<a href="https://colab.research.google.com/github/shirinlatifi/pe-project/blob/master/SG_MLP%26GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
pip install deap

In [36]:
import numpy as np
import tensorflow as tf
from deap import base, creator, tools, algorithms
import pandas as pd

In [37]:
df = pd.read_excel('oil SG.xlsx')

In [38]:
df

,T,P,Sgo
0,240.0,1439.07588,0.727671
1,240.0,1523.00000,0.728360
2,240.0,1623.00000,0.729221
3,240.0,1723.00000,0.730069
4,240.0,1823.00000,0.730905
...,...,...,...
499,NaN,NaN,NaN
500,NaN,NaN,NaN
501,NaN,NaN,NaN
502,NaN,NaN,NaN


In [39]:
df['P'] = df['P'].fillna(df['P'].mean())
df['T'] = df['T'].fillna(df['T'].mean())
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 479
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T       480 non-null    float64
 1   P       480 non-null    float64
 2   Sgo     480 non-null    float64
dtypes: float64(3)
memory usage: 15.0 KB


In [40]:
df.corr()

,T,P,Sgo
T,1.000000,0.006593,-0.483459
P,0.006593,1.000000,-0.093414
Sgo,-0.483459,-0.093414,1.000000


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [42]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('Sgo', axis=1), df['Sgo'], test_size=0.2, random_state=42)

In [45]:
# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [46]:
#Define the MLP Architecture
def build_mlp(hidden_units):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_train.shape[1],)))
    for units in hidden_units:
        model.add(tf.keras.layers.Dense(units, activation='relu'))
    model.add(tf.keras.layers.Dense(1))  # Output layer for regression
    return model

In [ ]:
build_mlp(3)

In [47]:
#Define the Fitness Function
def evaluate(individual):
    hidden_units = individual[0]  # Chromosome represents the hidden layer units
    mlp_model = build_mlp(hidden_units)
    mlp_model.compile(optimizer='adam', loss='mean_squared_error')
    mlp_model.fit(X_train, y_train, epochs=10, verbose=2)
    y_pred = mlp_model.predict(X_test)
    mse = np.mean(np.square(y_pred - y_test))
    return mse

In [ ]:
evaluate()

In [48]:
#print acc and MSE

In [49]:
#prediction

In [ ]:
#Visualization
import matplotlib.pyplot as plt

# Plotting the actual test data and predictions
plt.figure(figsize=(6,3))
plt.scatter(y_test, y_pred, color='blue', label='Predictions')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', linewidth=2, label='Ideal')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
plt.legend()
plt.grid(True)
# plt. ('mse or line function')
plt.show()

In [ ]:
#Set Up the Genetic Algorithm
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("hidden_units", np.random.randint, low=2, high=20, size=3)  #adjust the range and size
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.hidden_units)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=2, high=20, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [ ]:
#Create the Initial Population
population = toolbox.population(n=10)  #adjust the population size

#Genetic Algorithm Loop
ngen = 10  # Number of generations
for gen in range(ngen):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.3)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))

In [ ]:
#Extract Best Hyperparameters
best_individual = tools.selBest(population, k=1)[0]
best_hidden_units = best_individual[0]

In [ ]:
#Train the MLP with Best Hyperparameters
best_mlp_model = build_mlp(best_hidden_units)
best_mlp_model.compile(optimizer='adam', loss='mean_squared_error')
best_mlp_model.fit(X_train, y_train, epochs=100, verbose=1)

In [ ]:
#Evaluate on Test Set
y_pred = best_mlp_model.predict(X_test)
mse = np.mean(np.square(y_pred - y_test))
print("Mean Squared Error:", mse)

In [ ]:
# prediction

In [51]:
# acc score mse

In [52]:
#comparison before and after GA

In [53]:
# visualization